**Download Dataset dari API kaggle.com**

In [ ]:
! chmod 600 kaggle.json
! KAGGLE_CONFIG_DIR=/content/  kaggle datasets download -d ma7555/cat-breeds-dataset

Extract File Dataset

In [ ]:
import zipfile 
datasetZip = zipfile.ZipFile('/content/cat-breeds-dataset.zip')
datasetZip.extractall('/content/')

Mempersiapkan path folder

In [ ]:
import os
base_path = '/content/images'
cat_classes = os.listdir(base_path)
len(cat_classes)

**Image Preprocessing**

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    horizontal_flip = True,
    vertical_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest',
    width_shift_range = .2,
    height_shift_range = .2,
    validation_split = 0.2
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    base_path,
    target_size = (100,150),
    classes = cat_classes,
    class_mode = 'categorical',
    subset = 'training',
    shuffle = False
)

validation_generator = train_datagen.flow_from_directory(
    base_path,
    target_size = (100,150),
    classes = cat_classes,
    class_mode = 'categorical',
    subset = 'validation',
    shuffle = True
)

In [ ]:
print(train_generator[0])

In [ ]:
#
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}) :
    if(logs.get('accuracy') > 0.99) :
      print('\nAkurasi mencapai 99%')
      self.model.stop_training = True

callbacks = myCallback()

**Training Model**

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape = (100, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200, activation = 'relu'),
    tf.keras.layers.Dropout(0.3, seed=112),
    tf.keras.layers.Dense(500, activation = 'relu'),
    tf.keras.layers.Dropout(0.5, seed=112),
    tf.keras.layers.Dense(3, activation = 'softmax')

])

model.summary()

In [ ]:
model.compile(
    loss= 'sparse_categorical_crossentropy',
    optimizer = 'Adam',
    metrics = ['accuracy']
)


In [ ]:

result = model.fit(
    train_generator,
    steps_per_epoch = 20,
    epochs = 10,
    validation_steps = 5,
    validation_data = validation_generator,
    verbose = 1,
    batch_size=8,
    callbacks = [callbacks]
)



**Upload Image**

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
cat_classes

In [ ]:
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = tf.keras.utils.load_img(path, target_size=(100,150))
  imgplot = plt.imshow(img)
  x = tf.keras.utils.img_to_array(img)
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)

  class_list = os.listdir('images')
  
  print(fn)
  # if classes[0][0]==1:
  #   print('paper')
  # elif classes[0][1]==1:
  #   print('rock')
  # elif classes[0][2]==1:
  #   print('scissors')
  # else:
  #   print('unknown')
  for i in range(len(class_list)) :
    if classes[0][i] == 1. :
      print("Gambar terseteksi sebagai ", class_list[i])
      break